In [1]:
import pandas as pd
import os
import openai
prompt_data = pd.read_csv('./data/prompt_titles_cl.csv')

In [2]:
# retrieve rows from prompt_data where the value of the selected column is greater than 120 or less than 80. Return the data for the selected column and columns GPT_Name, CONCEPT, DOMAIN
def get_prompt_vars(prompt_data, column, concept):
    return prompt_data.loc[((prompt_data[column] > 120) | (prompt_data[column] < 80)) & (prompt_data['CONCEPT'] == concept), ['GPT_Name', 'CONCEPT', 'DOMAIN', column]]

In [3]:
# retrieve rows from prompt_data where the value of the selected column is greater than 120 or less than 80 and concept is equal to the user selection. Return a string combining the columns GPT_Name and user selected column
def get_prompt_string(prompt_data, column, concept):
    return prompt_data.loc[((prompt_data[column] > 120) | (prompt_data[column] < 80)) & (prompt_data['CONCEPT'] == concept), ['GPT_Name', column]].apply(lambda x: ' '.join(x), axis=1).tolist()    


In [4]:
prompt_data

,UniqueID,1_1,1_2,1_3,1_4,1_5,2_1,2_2,2_3,2_4,...,7_1,7_2,7_3,7_4,7_5,7_6,GPT_Name,Stub,CONCEPT,DOMAIN
0,B08301_010,146,25,18,34,15,92,318,138,681,...,76,1007,123,577,88,144,commutes to work using public transit public t...,Public transportation (excluding taxicab):,ECONOMY,Transport
1,B08303_002,47,67,62,58,132,88,99,215,90,...,78,33,61,45,49,104,commute length Less than 5 minutes,Less than 5 minutes,ECONOMY,Mobility
2,B08303_003,66,87,77,79,154,124,116,211,106,...,105,26,88,49,68,110,commute length 5 to 9 minutes,5 to 9 minutes,ECONOMY,Mobility
3,B08303_004,85,103,88,92,140,144,125,172,123,...,112,35,105,68,83,111,commute length 10 to 14 minutes,10 to 14 minutes,ECONOMY,Mobility
4,B08303_005,95,111,96,99,129,155,120,120,135,...,117,38,114,73,93,100,commute length 15 to 19 minutes,15 to 19 minutes,ECONOMY,Mobility
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,B16004_053,149,79,48,42,87,80,358,15,126,...,100,200,49,189,30,147,seniors who speak other indo-european language...,Speak other Indo-European languages (65+),POP,Language
207,B16004_058,279,124,89,111,47,75,345,14,128,...,144,283,57,211,104,140,seniors who speak asian and pacific island lan...,Speak Asian and Pacific Island languages (65+),POP,Language
208,B16004_063,132,87,67,34,78,92,242,22,109,...,139,217,134,178,62,199,seniors who speak other languages (65+),Speak other languages (65+),POP,Language
209,C02003_012,141,120,158,190,79,125,103,97,104,...,148,126,125,140,179,118,Two or more races,Population of two races:,POP,Race / Ancestory / Citizenship


In [5]:
prompt_data.columns

Index(['UniqueID', '1_1', '1_2', '1_3', '1_4', '1_5', '2_1', '2_2', '2_3',
       '2_4', '2_5', '3_1', '3_2', '3_3', '3_4', '3_5', '3_6', '3_7', '4_1',
       '4_2', '4_3', '4_4', '4_5', '5_1', '5_2', '5_3', '5_4', '5_5', '6_1',
       '6_2', '6_3', '6_4', '6_5', '6_6', '7_1', '7_2', '7_3', '7_4', '7_5',
       '7_6', 'GPT_Name', 'Stub', 'CONCEPT', 'DOMAIN'],
      dtype='object')

In [6]:
header = 'A person is trying to explain their neighborhood to a stranger. They present data comparing their neighborhood to the national average. A score of 100 means the neighborhood is equivalent to the national average, a score of 150 means the neighborhhood one and a half times the national average, a score of 200 means the neighborhood is twice the national average, a score of 50 means the neighborhood is half of the national average, a score of 300 means the neighborhood is three times the national average. Their neighborhood has the following charactersitics, described in #DATA# below. Data are presented for each charactersitic followed by a colon, and then a score. A good description of a neighborhood descrives characteristics that have scores which are greater than 120 or less than 80. \n\n#DATA#'
footer = '\n\nWrite a description of the neighborhood in no more than 200 words.'

In [7]:
def build_prompt(data, concept, a_class):
        data_string = ''
        #Select rows from prompt data where the column CONCEPT == concept and column name == class
        p_data = data.loc[data['CONCEPT'] == concept, ['GPT_Name', a_class]]

        for index, row in p_data.iterrows():
                data_string += row['GPT_Name'] + ': ' + str(row[a_class]) + '\n'
        
        
        return header + data_string + footer
    

In [8]:
prompt_data['CONCEPT'].unique()


array(['ECONOMY', 'ENVIRONMENT', 'POP'], dtype=object)

In [9]:
columns = [c for c in prompt_data.columns if c not in ['UniqueID', 'GPT_Name', 'Stub', 'CONCEPT', 'DOMAIN']]
concepts = prompt_data['CONCEPT'].unique()
openai.api_key = ""

responses = []
ctr = 0
for column in columns:
        ctr += 1
        for concept in concepts:
                ctr += 1
                print(ctr, column, concept)
                #get the prompt string for the column
                class_concept_prompt = build_prompt(prompt_data, concept, column)
                '''
                response = openai.Completion.create(
                        model="text-davinci-003",
                        prompt=class_concept_prompt,
                        temperature=0.3,
                        max_tokens=869,
                        top_p=1,
                        best_of=3,
                        frequency_penalty=0.2,
                        presence_penalty=0.28)'''
                responses.append({'class': column, 'domain': concept, 'prompt': class_concept_prompt})

responses_df = pd.DataFrame(responses)
responses_df.to_csv('./data/pompts.tsv', sep = '\t')

2 1_1 ECONOMY
3 1_1 ENVIRONMENT
4 1_1 POP
6 1_2 ECONOMY
7 1_2 ENVIRONMENT
8 1_2 POP
10 1_3 ECONOMY
11 1_3 ENVIRONMENT
12 1_3 POP
14 1_4 ECONOMY
15 1_4 ENVIRONMENT
16 1_4 POP
18 1_5 ECONOMY
19 1_5 ENVIRONMENT
20 1_5 POP
22 2_1 ECONOMY
23 2_1 ENVIRONMENT
24 2_1 POP
26 2_2 ECONOMY
27 2_2 ENVIRONMENT
28 2_2 POP
30 2_3 ECONOMY
31 2_3 ENVIRONMENT
32 2_3 POP
34 2_4 ECONOMY
35 2_4 ENVIRONMENT
36 2_4 POP
38 2_5 ECONOMY
39 2_5 ENVIRONMENT
40 2_5 POP
42 3_1 ECONOMY
43 3_1 ENVIRONMENT
44 3_1 POP
46 3_2 ECONOMY
47 3_2 ENVIRONMENT
48 3_2 POP
50 3_3 ECONOMY
51 3_3 ENVIRONMENT
52 3_3 POP
54 3_4 ECONOMY
55 3_4 ENVIRONMENT
56 3_4 POP
58 3_5 ECONOMY
59 3_5 ENVIRONMENT
60 3_5 POP
62 3_6 ECONOMY
63 3_6 ENVIRONMENT
64 3_6 POP
66 3_7 ECONOMY
67 3_7 ENVIRONMENT
68 3_7 POP
70 4_1 ECONOMY
71 4_1 ENVIRONMENT
72 4_1 POP
74 4_2 ECONOMY
75 4_2 ENVIRONMENT
76 4_2 POP
78 4_3 ECONOMY
79 4_3 ENVIRONMENT
80 4_3 POP
82 4_4 ECONOMY
83 4_4 ENVIRONMENT
84 4_4 POP
86 4_5 ECONOMY
87 4_5 ENVIRONMENT
88 4_5 POP
90 5_1 ECONOMY
9

In [10]:
responses_df

,class,domain,prompt
0,1_1,ECONOMY,A person is trying to explain their neighborho...
1,1_1,ENVIRONMENT,A person is trying to explain their neighborho...
2,1_1,POP,A person is trying to explain their neighborho...
3,1_2,ECONOMY,A person is trying to explain their neighborho...
4,1_2,ENVIRONMENT,A person is trying to explain their neighborho...
...,...,...,...
112,7_5,ENVIRONMENT,A person is trying to explain their neighborho...
113,7_5,POP,A person is trying to explain their neighborho...
114,7_6,ECONOMY,A person is trying to explain their neighborho...
115,7_6,ENVIRONMENT,A person is trying to explain their neighborho...


In [10]:

#responses_df['description'] = responses_df.apply(lambda x: openai.Completion.create(engine='text-davinci-003', prompt=x['prompt'], max_tokens=900, temperature=0.4, top_p=1, frequency_penalty=0, presence_penalty=0.6, stop=['\n\n']), axis=1)

In [11]:
get_prompt_vars(prompt_data, '1_1')
                response = openai.Completion.create(
                        model="text-davinci-003",
                        prompt=class_concept_prompt,
                        temperature=0.3,
                        max_tokens=869,
                        top_p=1,
                        best_of=3,
                        frequency_penalty=0.2,
                        presence_penalty=0.28)

IndentationError: unexpected indent (4185517605.py, line 2)